The aim of the notebook is to help you to take confidence with the directed graph class of NEMtropy. We will guide you through the functionality of the module and they can be use to reconstruct a network given partial information or to generate a null model given a certain network.
For more detail about the theory behind this, we suggest you to read .....

In [4]:
import numpy as np
import networkx as nx
from NEMtropy import DirectedGraph
from NEMtropy import matrix_generator as mg

According to the type of directed graph, the available models can be divided in:
 - models for binary network: "dcm" and "dcm_exp"
 - models for weighted networks: "crema", "decm" and "decm_exp"

We will make some examples about both classes of models, lets start from binary models. The use of "decm" and "decm_exp" models is deprecated.

# 1) Binary Models

In [31]:
# Lets generated a random directed adjacency matrix
# using matrix matrix generator module of NEMtropy
aux_adj = mg.random_binary_matrix_generator_custom_density(30,
                                                           0.2,
                                                           sym=False)

# then we convert it to numpy adjacency matrix, edgelist and degree sequence
edgelist = [(i,j) for i in range(aux_adj.shape[0]) for j in range(aux_adj.shape[0]) if aux_adj[i,j]!=0]
dseq_in = aux_adj.sum(axis=0)
dseq_out = aux_adj.sum(axis=1)

DirectedGraph instance can be initialised using adjacency matrix, edgelist or degree and strength sequences. As an example we initialise a graph instance using the adjacency matrix.

In [17]:
graph = DirectedGraph(aux_adj)

When you use an initiliasition instance that is not the adjacency matrix you must specify what you are passing to DirectedGraph. As an exaple we can use the edgelist.

In [19]:
graph = DirectedGraph(edgelist=edgelist)

## 1.1) Null Models

Now that our graph instance is initialised lets see what we can do with it. The first example that we consider is how to generate 10 (or more) random versions of our directed networl having (on average) the same degree sequence of the orginal one. These can be used to compare same properties of the original network with their expected values under a random null model preserving the degree sequence.

In [24]:
# Solve tool function solves, for the underlying graph instance, the selected "model" using "method" and "initial_guess" as starting point of the optimization.
# When the undirected network you are considering is binary the available model are 

graph.solve_tool(model="dcm_exp",
                 method="newton",
                 initial_guess="random")

# For networks that are not too large (>5000 nodes, but depends on the amount of ram of your laptop) we suggest to use "newton" method.
# You can che if the optimization process was successful by printing the error attribute. 
# Pay attention: error attribute is None if you didn't solve any model yet.

print("The largest error on the degree sequence is ", graph.error)

# As you can notice the error is quite small: the optimisation proccess worked properly!

The largest error on the degree sequence is  8.161116227256571e-11


In [28]:
# When your netwoork is too large, newton can be too costly in terms of memory and the suggested method is "fixed-point".
graph.solve_tool(model="dcm_exp",
                 method="fixed-point",
                 initial_guess="random")

print("The largest error on the degree sequence is ", graph.error)

# As you can notice the error is again quite small. For large networks can be better to look at the relative error on the degree sequence instead of the absolute one.

The largest error on the degree sequence is  3.090191214027982e-09


In [29]:
# After that we have solved the model we can use it to generate random versions of zachary karate club.
# The ensemble sampler function generates "n" random versions of zachary karate club using the model parameters computed by solve_tool.

graph.ensemble_sampler(10, cpu_n=2, output_dir="sample/")

# In the folder "sample/" there are 10 random copies of zachary karate club saved as edgelist.

## 1.2) Graph Reconstruction

Suppose now that you have only partial information about yournetwork: only the degree sequence is disclosed. Following the same procedure described above, we can generate generate a network satisfying (on average) the observed degree sequence.

In [32]:
# We initialise the graph instance using dseq
# Don't forget to concatenate out-degree and in-degree
# sequences before passing it. The order is important [out,in]
graph = DirectedGraph(degree_sequence=np.concatenate([dseq_out,dseq_in]))

# We solve the binary model
graph.solve_tool(model="dcm_exp",
                 method="newton",
                 initial_guess="random")

# We generate an ensemble copy
graph.ensemble_sampler(1, cpu_n=2, output_dir="sample/")

# We read the edgelist
edgelist_ens = np.loadtxt("sample/0.txt")

# And generate a networkx graph from it
G = nx.DiGraph()
G.add_edges_from(edgelist_ens)

### ADD network draw

# 2) Weigthed Models

## 2.1) Null Models

In [36]:
# First of all we generate the adjacency matrix of a weighted undirected graph using the matrix_generator module of NEMtropy
adj_weigh = mg.random_weighted_matrix_generator_uniform_custom_density(n=100,
                                                                                     p=0.2,
                                                                                     sym=False,
                                                                                     sup_ext=30,
                                                                                     intweights=True)

# Then we intialise the graph instance
graph = DirectedGraph(adj_weigh)

# We can the see the graph sequence of the graph by tapping
print("Out strength sequence ",graph.out_strength)
print("In strength sequence ",graph.in_strength)

Out strength sequence  [139. 234. 197. 252. 448. 330. 263. 291. 287. 300. 375. 317. 471. 300.
 328. 309. 331. 230. 348. 200. 314. 338. 338. 407. 238. 260. 233. 346.
 337. 240. 307. 295. 280. 319. 354. 363. 331. 280. 381. 330. 349. 335.
 314. 282. 354. 356. 236. 322. 396. 353. 172. 296. 211. 306. 232. 271.
 385. 302. 393. 182. 220. 179. 364. 314. 300. 313. 312. 281. 382. 291.
 264. 333. 451. 351. 345. 404. 356. 333. 470. 291. 231. 313. 286. 316.
 258. 273. 356. 298. 367. 266. 213. 326. 231. 308. 301. 475. 355. 388.
 185. 183.]
In strength sequence  [351. 286. 243. 362. 403. 229. 277. 332. 281. 354. 232. 313. 352. 273.
 399. 277. 265. 352. 271. 352. 290. 390. 171. 263. 323. 403. 254. 233.
 358. 307. 285. 428. 310. 377. 218. 280. 329. 362. 311. 336. 290. 353.
 243. 202. 367. 282. 288. 293. 351. 341. 363. 216. 186. 388. 420. 299.
 125. 179. 406. 298. 254. 260. 362. 287. 402. 406. 255. 292. 273. 390.
 287. 348. 315. 357. 299. 342. 300. 205. 376. 358. 275. 260. 246. 426.
 272. 341. 279. 344.

In [38]:
# Similarly to the binary case, we can solve the weighted model by solve_tool
graph.solve_tool(model="crema",
                 method="newton",
                 initial_guess="random",
                 adjacency="dcm_exp",
                 method_adjacency="newton")

# The optimisation problem is divided in two steps: firstly the binary model is solved and its solution are one
# of the inputs for the weighted model solver. You can choose the method used for the binary and weighted part separately.
# You can even pass an adjacency matrix as imput for the binary part and we will go through this in the next subsection.

In [39]:
# Lets check if the optimisation was successful by printing the errors on the degree and strength sequences
print(graph.error_degree, graph.error_strength)

9.059863970151127e-10 2.5487736365903402e-08


In [ ]:
# Similarly to the binary case, we can generate 10 ensembles copies using ensemble_sampler
graph.ensemble_sampler(10, cpu_n=2, output_dir="sample/")

## 2.2) Reconstruction

Suppose you have only partial information about the network, for example:
 - The binary structure is well-known and you want to assign weights to network edges;
 - Only marginals information are available: degree and strength sequences.

You can use NEMtropy methods to reconstruct a network configuration that is consistent with the available informations.

### 2.2.1) Adjacency matrix 

In [44]:
# For this example we generate the adjacency matrix of a weighted directed graph using the matrix_generator module of NEMtropy
adj_weigh = mg.random_weighted_matrix_generator_uniform_custom_density(n=100,
                                                                                     p=0.2,
                                                                                     sym=False,
                                                                                     sup_ext=30,
                                                                                     intweights=True)

# Lets store the binary structure and strength sequence
adj_bin = adj_weigh.astype(bool).astype(float)
out_strength = adj_weigh.sum(axis=1)
in_strength = adj_weigh.sum(axis=0)

# Now we suppose that the weighted structure is not available
del adj_weigh

# We can initialiase our graph instance with the strength sequence
graph_weighted = DirectedGraph(strength_sequence=np.concatenate([out_strength, in_strength]))

In [45]:
# Then we solve the model using the adjacency binary matrix as an input for the solver
graph_weighted.solve_tool(model="crema",
                          method="newton",
                          initial_guess="random",
                          adjacency=adj_bin)

In [46]:
# Lets check if the optimisation was successful by printing the error on strength sequence
print(graph_weighted.error_strength)

1.7053025658242404e-13


In [ ]:
# Finally we generate a network having the binary structure of "adj_bin" and 
# and weights satisfying (on average) the given "strength_seq"
graph.ensemble_sampler(1, cpu_n=2, output_dir="sample/")

### 2.2.2) Only the marginals are available

In [48]:
# As an example we generate the adjacency matrix of a weighted directed graph using the matrix_generator module of NEMtropy
adj_weigh = mg.random_weighted_matrix_generator_uniform_custom_density(n=100,
                                                                                     p=0.2,
                                                                                     sym=False,
                                                                                     sup_ext=30,
                                                                                     intweights=True)

# Lets store the binary structure and strength sequence
out_degree = adj_weigh.astype(bool).astype(float).sum(axis=1)
in_degree = adj_weigh.astype(bool).astype(float).sum(axis=0)
out_strength = adj_weigh.sum(axis=1)
in_strength = adj_weigh.sum(axis=0)

# Now we suppose that the weighted structure is not available
del adj_weigh

# We can initialiase our graph instance with degree and strength sequence
graph_weighted = DirectedGraph(degree_sequence=np.concatenate([out_degree, in_degree]),
                               strength_sequence=np.concatenate([out_strength, in_strength]))

In [51]:
# We solve the model, we pass a model as adjacency and the solver will use degree sequence
# to infer the binary structure
graph_weighted.solve_tool(model="crema",
                          method="newton",
                          initial_guess="random",
                          adjacency="dcm_exp",
                          method_adjacency="newton")

In [52]:
# Lets check if the optimisation was successful by printing the errors on strength and degree sequences
print(graph_weighted.error_degree, graph_weighted.error_strength)

2.1316282072803006e-14 0.0003549837721266158


In [ ]:
# Now we generate a network having a binary structure inferred from dseq
# and weights satisfying (on average) the given "strength_seq"
graph.ensemble_sampler(1, cpu_n=2, output_dir="sample/")

### ADD the part were we read the network and plot it